# dim_produtos

Esse notebook é responsável por criar uma tabela com as informações básicas dos produtos do e-commerce.


## Configurações Iniciais

### Importando as Bibliotecas

In [0]:
from pyspark.sql import functions as F

### Carregando os Dados

In [0]:
df_products_dataset = (
    spark.read.csv(
        "/Volumes/workspace/default/e_commerce/olist_products_dataset.csv",
        header=True,
        inferSchema=True
    )
    .withColumn(
        "product_category_name",
        F.regexp_replace(
           F.upper(F.col("product_category_name")),
           "_",
           " "
       )
    )
)

## Transformação dos Dados

In [0]:
df_conversions = (
    df_products_dataset
    .withColumn(
        "product_weight_kg",
        F.col("product_weight_g") / 1000
    )
    .withColumn(
        "product_length_m",
        F.col("product_length_cm") / 100
    )
    .withColumn(
        "product_height_m",
        F.col("product_height_cm") / 100
    )
    .withColumn(
        "product_width_m",
        F.col("product_width_cm") / 100
    )
)

In [0]:
df_final_products_dataset = (
    df_conversions
    .withColumn(
        "product_volume",
        F.col("product_length_m")
        * F.col("product_height_m")
        * F.col("product_width_m")
    )
    .select(
        "product_id",
        "product_category_name",
        "product_weight_kg",
        "product_length_m",
        "product_height_m",
        "product_width_m",
        "product_volume"
    )
)

## Salvamento dos Dados

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ifrs_dev_ecommerce.gold.dim_produtos (
    product_id STRING PRIMARY KEY COMMENT "ID do produto",
    product_category_name STRING COMMENT "Nome da categoria do produto",
    product_weight_kg DOUBLE COMMENT "Peso do produto (kg)",
    product_length_m DOUBLE COMMENT "Comprimento do produto (m)",
    product_height_m DOUBLE COMMENT "Altura do produto (m)",
    product_width_m DOUBLE COMMENT "Largura do produto (m)",
    product_volume DOUBLE COMMENT "Volume do produto"
)
USING DELTA
CLUSTER BY (product_id, product_category_name)
TBLPROPERTIES ("quality"="gold", "layer"="data-engineering")
COMMENT "Dimensão de produtos contendo informações de identificação, categoria e medidas.";

-- Adicionando tags à tabela
ALTER TABLE ifrs_dev_ecommerce.gold.dim_produtos SET TAGS ("quality"="gold", "layer"="data-engineering");

In [0]:
(
    df_final_products_dataset
    .write.mode("overwrite")
    .format("delta")
    .saveAsTable("ifrs_dev_ecommerce.gold.dim_produtos")
)